In [15]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
np.unique(cv2.imread("/home/sevashasla/coding/datasets/replica/Replica_Dataset/room_0/Sequence_1/semantic_class/semantic_class_0.png", cv2.IMREAD_UNCHANGED))

array([ 0, 11, 12, 13, 20, 29, 31, 40, 44, 47, 59, 60, 63, 64, 65, 76, 80,
       92, 93, 97, 98], dtype=uint8)

In [3]:
np.loadtxt("/home/sevashasla/coding/datasets/replica/Replica_Dataset/room_0/Sequence_1/traj_w_c.txt", delimiter=" ").reshape(-1, 4, 4)[0]

array([[-0.3837331 ,  0.05730035,  0.92166457,  1.81724055],
       [-0.92344405, -0.02381091, -0.38299365,  2.11536135],
       [-0.        , -0.998073  ,  0.0620507 , -0.44625386],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [14]:
torch.arange(10)[torch.randn((10,)) > 0]

tensor([0, 3, 6, 7, 9])

In [24]:
torch.ones(3, dtype=torch.long).dtype

torch.int64

In [25]:
dict.fromkeys('aboba')

{'a': None, 'b': None, 'o': None}

In [34]:
import yaml
from types import SimpleNamespace

In [42]:
with open('/home/sevashasla/coding/nerf/ngp_with_semantic_nerf/configs/try_config.yaml') as f:
    g = yaml.safe_load(f)
g['semantic']

{'num_semantic_classes': None, 'semantic_remap': None}